## Import

In [1]:
import numpy as np
import pandas as pd

## Exploring data

### Read raw data from file

In [2]:
raw_df = pd.read_csv("../data/raw/data.csv")

In [3]:
raw_df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2300,s2301,Movie,The Perfect Picture: Ten Years Later,Shirley Frimpong-Manso,"Jackie Appiah, Naa Ashorkor Mensa-Doku, Lydia ...",Ghana,"July 3, 2020",2019,TV-MA,148 min,"Comedies, Dramas, International Movies","From flawed husbands to shaky finances, new co..."
6397,s6398,Movie,Cabin Fever,Travis Zariwny,"Gage Golightly, Matthew Daddario, Samuel Davis...",United States,"June 6, 2019",2016,R,98 min,"Horror Movies, Thrillers",As a flesh-eating virus rips its way through a...
2941,s2942,TV Show,Locke & Key,NaN,"Darby Stanchfield, Connor Jessup, Emilia Jones...","Canada, United States","February 7, 2020",2020,TV-14,1 Season,"TV Action & Adventure, TV Dramas, TV Mysteries","After their dad's murder, three siblings move ..."
4935,s4936,Movie,Amateur,Ryan Koo,"Michael Rainey Jr., Josh Charles, Brian White,...",United States,"April 6, 2018",2018,TV-MA,96 min,"Dramas, Independent Movies, Sports Movies","After he's recruited to an elite prep school, ..."
2549,s2550,Movie,John Henry,Will Forbes,"Terry Crews, Ludacris, Jamila Velazquez, J.J. ...",United States,"May 9, 2020",2020,R,92 min,"Dramas, Thrillers",A reformed LA gang member upends his peaceful ...


We could drop the show_id and description column because they are not useful for our analysis.

In [4]:
raw_df.drop(columns = ['show_id', 'description'], inplace = True)

### How many rows and how many columns does the raw data have?

In [5]:
shape = raw_df.shape
print(f"Data has {shape[0]} rows and {shape[1]} columns")

Data has 8807 rows and 10 columns


### What is the meaning of each row?

Each row represents information about a movie/ tv show on Netflix from 2008 to mid 2021.

### Are there duplicated rows?

In [6]:
unique_rows = raw_df.drop_duplicates().shape[0] - shape[0]
print(f'Number of duplicate rows: {unique_rows}')

Number of duplicate rows: 0


### What is the meaning of each column?

| Column Name | Description |
| --- | --- |
| show_id | Unique ID for each movie/ tv show |
| type | Type of the movie/ tv show (Movie or TV Show) |
| title | Title of the movie/ tv show |
| director | Director of the movie/ tv show |
| cast | Cast of the movie/ tv show |
| country | Country where the movie/ tv show was produced |
| date_added | Date when the movie/ tv show was added to Netflix |
| release_year | Year when the movie/ tv show was released |
| rating | Rating of the movie/ tv show |
| duration | Duration of the movie/ tv show |
| listed_in | Genre of the movie/ tv show |
| description | Description of the movie/ tv show |

### What is the current data type of each column? Are there columns having inappropriate data types?


In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8807 non-null   object
 1   title         8807 non-null   object
 2   director      6173 non-null   object
 3   cast          7982 non-null   object
 4   country       7976 non-null   object
 5   date_added    8797 non-null   object
 6   release_year  8807 non-null   int64 
 7   rating        8803 non-null   object
 8   duration      8804 non-null   object
 9   listed_in     8807 non-null   object
dtypes: int64(1), object(9)
memory usage: 688.2+ KB


We have null values in director, cast, country, date_added, rating and duration columns.

The reason why we don't want to drop the null values in country column is because it could provide some interesting points.

In [8]:
raw_df['director'] = raw_df['director'].fillna('Unknown')
raw_df['cast'] = raw_df['cast'].fillna('Unknown')
raw_df['country'] = raw_df['country'].bfill()

raw_df.dropna(inplace=True)

In [9]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8790 entries, 0 to 8806
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          8790 non-null   object
 1   title         8790 non-null   object
 2   director      8790 non-null   object
 3   cast          8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 755.4+ KB


### Convert data type of date_added column to datetime

In [10]:
raw_df['date_added'] = pd.to_datetime(raw_df['date_added'].str.strip())
raw_df['year_added'] = raw_df['date_added'].dt.year
raw_df['month_added'] = raw_df['date_added'].dt.month

### With each numerical column, how are values distributed?

- Percentage of missing values?

In [11]:
raw_df.select_dtypes('number').isna().sum() / len(raw_df)

release_year    0.0
year_added      0.0
month_added     0.0
dtype: float64

- Describe the values.

In [12]:
raw_df.select_dtypes('number').describe()

,release_year,year_added,month_added
count,8790.000000,8790.000000,8790.000000
mean,2014.183163,2018.873606,6.655859
std,8.825466,1.573568,3.436103
min,1925.000000,2008.000000,1.000000
25%,2013.000000,2018.000000,4.000000
50%,2017.000000,2019.000000,7.000000
75%,2019.000000,2020.000000,10.000000
max,2021.000000,2021.000000,12.000000


All values seem to be normal.

### With each categorical column, how are values distributed?

### Missing ratios of categorical columns

In [13]:
df_copy = raw_df.copy()
df_copy = df_copy.drop(axis = 1, columns=['date_added', 'release_year', 'year_added', 'month_added'])
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def num_values(s):
    s = s.str.split(',')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.str.split(',')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = df_copy.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,type,title,director,cast,country,rating,duration,listed_in
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
num_values,2,8922,5119,39249,197,14,220,73
value_ratios,"{'Movie': 69.7, 'TV Show': 30.3}","{'Love': 0.1, ' Explained': 0.1, 'Hello': 0.0,...","{'Unknown': 27.3, 'Rajiv Chilaka': 0.2, 'Raúl ...","{'Unknown': 1.3, ' Anupam Kher': 0.1, ' Rupa B...","{'United States': 31.9, 'India': 9.8, 'United ...","{'TV-MA': 36.5, 'TV-14': 24.5, 'TV-PG': 9.8, '...","{'1 Season': 20.4, '2 Seasons': 4.8, '3 Season...","{' International Movies': 13.6, 'Dramas': 8.3,..."


### Save processed data

In [14]:
raw_df.to_csv('../data/processed/data.csv')